<a href="https://colab.research.google.com/github/yuki-tamaribuchi/ml_code_kata/blob/master/kaggle_titanic_lightgbm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir /root/.kaggle
!cp drive/'My Drive'/kaggle/kaggle.json /root/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [2]:
!kaggle competitions download -c titanic

test.csv: Skipping, found more recently modified local copy (use --force to force download)
train.csv: Skipping, found more recently modified local copy (use --force to force download)
gender_submission.csv: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split

In [4]:
train_df=pd.read_csv('train.csv')
test_df=pd.read_csv('test.csv')

In [5]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
train_df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [7]:
sex_dummy=pd.get_dummies(train_df['Sex'])
pclass_dummy=pd.get_dummies(train_df['Pclass'],prefix='Pclass')
embarked_dummy=pd.get_dummies(train_df['Embarked'],prefix='Emb')

train_df=train_df.drop(['Ticket','Cabin','Name','Sex','Pclass','Embarked','Cabin','PassengerId'],axis=1)
train_df=pd.concat([train_df,sex_dummy,pclass_dummy,embarked_dummy],axis=1)

train_df['Age']=train_df['Age'].fillna(train_df['Age'].mean())

In [8]:
train_df.isnull().sum()

Survived    0
Age         0
SibSp       0
Parch       0
Fare        0
female      0
male        0
Pclass_1    0
Pclass_2    0
Pclass_3    0
Emb_C       0
Emb_Q       0
Emb_S       0
dtype: int64

In [9]:
y_train_df=train_df['Survived']
X_train_df=train_df.drop(['Survived'],axis=1)

In [10]:
test_df.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [11]:
sex_dummy=pd.get_dummies(test_df['Sex'])
pclass_dummy=pd.get_dummies(test_df['Pclass'],prefix='Pclass')
embarked_dummy=pd.get_dummies(test_df['Embarked'],prefix='Emb')

test_df=test_df.drop(['Ticket','Cabin','Name','Sex','Pclass','Embarked','Cabin','PassengerId'],axis=1)
test_df=pd.concat([test_df,sex_dummy,pclass_dummy,embarked_dummy],axis=1)

test_df['Age']=test_df['Age'].fillna(test_df['Age'].mean())
test_df['Fare']=test_df['Fare'].fillna(test_df['Fare'].mean())

X_test=test_df

In [12]:
X_test.isnull().sum()

Age         0
SibSp       0
Parch       0
Fare        0
female      0
male        0
Pclass_1    0
Pclass_2    0
Pclass_3    0
Emb_C       0
Emb_Q       0
Emb_S       0
dtype: int64

In [13]:
X_train,X_test,y_train,y_test=train_test_split(X_train_df,y_train_df)

In [14]:
import lightgbm as lgb

In [15]:
lgb_train=lgb.Dataset(X_train,y_train)
lgb_eval=lgb.Dataset(X_test,y_test,reference=lgb_train)

In [16]:
lgbm_params={'objective':'multiclass','num_class':3}

In [17]:
model=lgb.train(lgbm_params,lgb_train,valid_sets=lgb_eval)

[1]	valid_0's multi_logloss: 0.628498
[2]	valid_0's multi_logloss: 0.594682
[3]	valid_0's multi_logloss: 0.56803
[4]	valid_0's multi_logloss: 0.547325
[5]	valid_0's multi_logloss: 0.530171
[6]	valid_0's multi_logloss: 0.515025
[7]	valid_0's multi_logloss: 0.502544
[8]	valid_0's multi_logloss: 0.492145
[9]	valid_0's multi_logloss: 0.485113
[10]	valid_0's multi_logloss: 0.475983
[11]	valid_0's multi_logloss: 0.468613
[12]	valid_0's multi_logloss: 0.463626
[13]	valid_0's multi_logloss: 0.459112
[14]	valid_0's multi_logloss: 0.457296
[15]	valid_0's multi_logloss: 0.455131
[16]	valid_0's multi_logloss: 0.452827
[17]	valid_0's multi_logloss: 0.451752
[18]	valid_0's multi_logloss: 0.451117
[19]	valid_0's multi_logloss: 0.449215
[20]	valid_0's multi_logloss: 0.449565
[21]	valid_0's multi_logloss: 0.45109
[22]	valid_0's multi_logloss: 0.452718
[23]	valid_0's multi_logloss: 0.45488
[24]	valid_0's multi_logloss: 0.455524
[25]	valid_0's multi_logloss: 0.457023
[26]	valid_0's multi_logloss: 0.46000

In [18]:
y_pred=model.predict(X_test,num_iteration=model.best_iteration)
y_pred_max=np.argmax(y_pred,axis=1)

In [19]:
accuracy=sum(y_test==y_pred_max)/len(y_test)
print(accuracy)

0.8161434977578476
